In [49]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [25]:
data=pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [3]:
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [5]:
data.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

In [34]:
categorical=[]
for c,value in data.iteritems():
    if value.dtype=='object':
        categorical.append(c)
        
numerical=data.columns.difference(categorical)
numerical
categorical

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [26]:
def map_attrition(x):
    if x=='Yes':
        return 1
    elif x=='No':
        return 0
data['Attrition']=data.Attrition.apply(lambda x: map_attrition(x))

In [27]:
data.Attrition.value_counts()

0    1233
1     237
Name: Attrition, dtype: int64

In [35]:
data_dummy=pd.get_dummies(data)

In [37]:
data_dummy.shape


(1470, 56)

In [38]:
data_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 56 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Age                                1470 non-null   int64
 1   Attrition                          1470 non-null   int64
 2   DailyRate                          1470 non-null   int64
 3   DistanceFromHome                   1470 non-null   int64
 4   Education                          1470 non-null   int64
 5   EmployeeCount                      1470 non-null   int64
 6   EmployeeNumber                     1470 non-null   int64
 7   EnvironmentSatisfaction            1470 non-null   int64
 8   HourlyRate                         1470 non-null   int64
 9   JobInvolvement                     1470 non-null   int64
 10  JobLevel                           1470 non-null   int64
 11  JobSatisfaction                    1470 non-null   int64
 12  MonthlyIncome       

In [39]:
df_train,df_test=train_test_split(data_dummy,train_size=.7,random_state=42)


In [45]:
X_train=df_train[data_dummy.columns.difference(['Attrition'])]
y_train=df_train['Attrition']

X_test=df_test[data_dummy.columns.difference(['Attrition'])]
y_test=df_test['Attrition']

In [65]:
xgb_cfl=xgb.XGBClassifier(n_jobs=-1,objective='binary:logistic')
xgb_cfl.get_params()

xgb_model=xgb_cfl.fit(X_train,y_train)

In [70]:
y_test_pred=xgb_model.predict(X_test)
accuracy_score(y_test_pred,y_test)

0.8707482993197279

In [72]:
params={'n_estimators':[100,200,300,400,500],
       'learning_rate':[.01,.02,.05,.1,.25],
       'min_child_weight':[1,5,7,9,10],
       'gamma':[.1,.5,1,1.5,5],
       'subsample':[.5,.6,.8,1],
       'colsample_bytree':[.6,.8,1.0],
       'max_depth':[3,4,5,6,7,10,12]}

folds=5
param_comb=800
random_search=RandomizedSearchCV(xgb_cfl, param_distributions=params,n_iter=param_comb, scoring='accuracy',cv=5)

In [73]:
random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           callbacks=None, colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=0, gpu_id=-1,
                                           grow_policy='depthwise',
                                           importance_type=None,
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_bin=256,...
                                           num_parallel_tree=1,
                                           predictor='auto', random_state=0,
                                           reg_alpha=0, reg_lambda=1, ...),
                   n_iter=800,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.1, 0.5, 1, 1.5, 5],
                                        'learning_rate': [0.01, 0.02, 0.05, 0.1,
                                                          0.25],
                                        'max_depth': [3, 4, 5, 6, 7, 10, 12],
                                        'min_child_weight': [1, 5, 7, 9, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.5, 0.6, 0.8, 1]},
                   scoring='accuracy')

In [75]:
random_search.best_params_

{'subsample': 0.6,
 'n_estimators': 100,
 'min_child_weight': 7,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 0.5,
 'colsample_bytree': 0.6}

In [117]:
xgb_hyp1=xgb.XGBClassifier(n_job=-1,subsample=0.6, n_estimators=100,min_child_weight= 9,max_depth= 7,learning_rate=0.1,gamma=0.5,colsample_bytree= 0.6)
xgb_hyp1.fit(X_train,y_train)

[21:08:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "n_job" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.6,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=9,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_job=-1, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [74]:
xgb_pred=random_search.predict(X_test)
accuracy_score(y_test,xgb_pred)

0.8662131519274376

In [118]:
xgbhyp1_pred=xgb_hyp1.predict(X_test)
accuracy_score(y_test,xgbhyp1_pred)

0.8820861678004536

In [69]:
xgb_model.feature_importances_

array([0.02040272, 0.01445948, 0.02901643, 0.01931156, 0.01488   ,
       0.        , 0.00715761, 0.0175041 , 0.02255406, 0.00870343,
       0.        , 0.02037333, 0.03973878, 0.01926368, 0.03695508,
       0.01815944, 0.        , 0.01709219, 0.0255718 , 0.01848092,
       0.        , 0.0172861 , 0.02296992, 0.03842295, 0.01171557,
       0.00901133, 0.00869358, 0.        , 0.        , 0.        ,
       0.02865932, 0.04095873, 0.021897  , 0.01626282, 0.01209192,
       0.0124609 , 0.04944135, 0.03808333, 0.01743365, 0.02621426,
       0.        , 0.05430362, 0.        , 0.01097997, 0.        ,
       0.01853008, 0.        , 0.04010495, 0.03124563, 0.01310053,
       0.01754284, 0.02837824, 0.01845841, 0.03105048, 0.01507782],
      dtype=float32)

In [105]:
params={
       'gamma':[.1,.2,.3,.4,.5,.6,.7,.8]}

folds=5
grid_search=GridSearchCV(xgb_cfl, param_grid=params, scoring='accuracy',cv=5)

In [106]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     callbacks=None, colsample_bylevel=1,
                                     colsample_bynode=1, colsample_bytree=1,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=0, gpu_id=-1,
                                     grow_policy='depthwise',
                                     importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.300000012, max_bin=256,
                                     max_cat_to_onehot=4, max_delta_step=0,
                                     max_depth=6, max_leaves=0,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                                     n_estimators=100, n_jobs=-1,
                                     num_parallel_tree=1, predictor='auto',
                                     random_state=0, reg_alpha=0, reg_lambda=1, ...),
             param_grid={'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]},
             scoring='accuracy')

In [107]:
grid_search.best_params_

{'gamma': 0.1}

In [108]:
y_pred_gd=grid_search.predict(X_test)
accuracy_score(y_test,y_pred_gd)

0.8775510204081632